In [1]:
import ipyrad.analysis as ipa
import ipcoal
import toytree
import toyplot
import pandas as pd
from ipyrad.analysis.baba21 import Drawing

In [2]:
import random

def generate_partitioned_tests_from_names(sources, targets_raw, outgroup, no_repeat=True):
    '''This method is special only for partitioned D-statistics'''
    
    if type(sources) is not list:
        sources = [sources]
    
    if type(outgroup) is not list:
        outgroup = [outgroup]
    
    #Declare empty result
    tests =[]
    
    for source in sources:
        #If in sources are the outgroup skip it
        if source in outgroup:
            continue
        
        
        #New unlinked targets list every loop
        targets = targets_raw.copy()
            
        #Remove source and outgroups in case they are repeated in the targets list
        if source in targets:
            targets.remove(source)
        for i in outgroup:
            if i in targets:
                targets.remove(i) 

        #declare empty lists 
        included = []
        

        #Iterate over all targets
        for i, _ in enumerate(targets):
            #empty temp p variables
            p1 = [] 
            p2 = []
            source2 = []

           #only do following if target was not already used (not in included)
            if targets[i] not in included:
                #use the first target as p1
                p1 = [targets[i]]
                #mark it is used appending to the list
                if no_repeat: included.append(targets[i])
                #avoid out of boundary errors
                if i < len(targets)-1:
                    #use the next target as p2
                    if targets[i+1] not in included:
                        p2 = [targets[i+1]]
                        #mask it as used
                        if no_repeat: included.append(targets[i+1])
                else:
                    #in case it is the last element (for odd number of targets, use a random one (but the current) as p2)
                    targets.remove(targets[i])
                    p2 =  [random.choice(targets)]
                
                if i < len(targets)-2:
                    if targets[i+2] not in included:
                        source2 = [targets[i+2]]
                        if no_repeat: included.append(targets[i+1])
                else:
                    targets.remove(targets[i])
                    source2 =  [random.choice(targets)]

                #append test to the return list
                tests.append({'p1': p1, 'p2': p2, 'p3_1': [source], 'p3_2': source2, 'p4': outgroup})
                
    return tests #return list of dictionaries with every test

In [ ]:
## Test abbas using partitioned

##['70', '5', '18', [('r7', 'r15'), ('r7', 'r17'), ('r17', 'r6'), ('r18', 'r5')]]
import os
arr = os.listdir("/home/carlos/AutoABBA/scene_1/")

tips=20
tree = toytree.rtree.baltree(ntips=tips, treeheight=10e6)

final_result = []

for hdf5 in arr: 
    
    trueSource, trueTarget = hdf5.split("_")[2].split(".")[0].split("to")
    test_id = hdf5.split("_")[1]
    
    print (test_id)
    
    baba = ipa.baba21(f"/home/carlos/AutoABBA/scene_1/{hdf5}")
    source = tree.get_tip_labels()
    targets = tree.get_tip_labels()
    outgroup = "r19"

    tests = generate_partitioned_tests_from_names(source, targets, outgroup, no_repeat=False)
    
    result = []
    for i in tests:
        try:
#             df = baba.run_test(i, nboots=100, quiet=True)
            df = baba.run_partitioned_test(i, nboots=100, quiet=True)
        except:
            continue
        
        if df["Z12"].values[0] > 2.5:
            pair = (*i["p3_1"], *get_significant_donee(df, i))
            if pair not in result:
                result.append(pair)



    
#     
#     print([test_id, trueSource, trueTarget, result])
#     final_result.append([test_id, trueSource, trueTarget, result])
#     print(result)
#     with open("fulldepur.txt", "a") as f:
#         f.write(str([test_id, trueSource, trueTarget, result])+ "\n")
    file = open("purePartitioned_abbababa.txt", "a")
    file.write(str([test_id, trueSource, trueTarget, result])+ "\n")
    file.close()

18


In [18]:
tips=20
tree = toytree.rtree.baltree(ntips=tips, treeheight=10e6)
tree.draw(ts="p")

(<toyplot.canvas.Canvas at 0x7f30004264d0>,
 <toytree.Render.ToytreeMark at 0x7f300013b8d0>)

<svg class="toyplot-canvas-Canvas" height="265.0px" id="t4f8d0756000d48aba37d7b2e52389099" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 360.0 265.0" width="360.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11 r12 r13 r14 r15 r16 r17 r18 r19 0 5000000 10000000

In [29]:
baba = ipa.baba21(f"/home/carlos/AutoABBA/scene_1/test-baba-miss50_18_4to18.snps.hdf5")

test = {
    'p1': ['r7'], 
    'p2': ['r18'], 
    'p3_1': ['r3'], 
    'p3_2': ['r4'], 
    'p4': ['r19']
    }

df = baba.run_partitioned_test(test, 
    nboots=100, 
    quiet=True
)

result = []
if df["Z12"].values[0] > 2.5:
        pair = ("r3", *get_significant_donee(df, test))
        if pair not in result:
            result.append(pair)
result

[('r3', 'r7')]

In [ ]:
def get_significant_donee_from_Dpart(df_result, test):
    if df_result["D1"].values[0] > 0:
            significative_donee = test["p2"]
    elif df_result["D1"].values[0] < 0:
        significative_donee = test["p1"]
    return significative_donee